# Symbolic reduction of given models
Passes a symbolic parameter and a data vector to the model function, then given the velocity vector at the boundary, figures out a model with a reduce number of parameters

In [4]:
# Includes
using SymPy

## Subfunctions

In [28]:
# Assign a vector of symbolic variables

# Assign a single variable, given it's string name and input
function string_as_varname(s::AbstractString,v::Any)
    s=symbol(s) 
    @eval (($s) = ($v))
end

# Assign a matrix of symbolic variables to string s (the variables share the same name)
function string_as_symb_vec(s::AbstractString, dims::Tuple)
    string_as_varname(s, [string_as_varname("$s$(i1)",Sym("$s$(i1)")) for i1 in 1:prod(dims)])
    string_as_varname(s, reshape(eval(symbol(s)), dims))
end

string_as_symb_vec (generic function with 1 method)

In [17]:
# Example assignment
string_as_symb_vec("phi", (3,4))

3x4 Array{Any,2}:
 phi1  phi4  phi7  phi10
 phi2  phi5  phi8  phi11
 phi3  phi6  phi9  phi12

In [ ]:
# Given the normal at the boundary, substitute parameter combinations
function phi_combine(y_symb::SymFunction, v::AbstractArray)
    
    
    
    return y_comb, phi_comb
end

In [ ]:
# Take the limits of the combined model
function take_limits(y_comb::SymFunction, v::AbstractArray)
    
    
    
    return y_lims
end

In [ ]:
# Substitute back parameter combinations
function phi_reduce(y_lims::SymFunction, phi_comb)
    
    
    return y_red, phi_red, phi_red_mapping
end

In [ ]:
# Turn the symbolic model into a callable function
function create_reduced_model(y_red::SymFunction)
    # model_red = lambdify(y_red) # Roughly, with correct wrappers such that model_red(phi, x)
    
    return model_red
end

In [ ]:
# Compute the reduced parameter vector numerically
function numeric_reduction(phi_boundary::AbstractArray, phi_red_mapping::AbstractArray)
    # basically do a general BK_reduce function
    
    return phi_red_numeric
end

## Main function

In [32]:
# Reduces the model described in the function model(phi::parameters, x::evaluation_locations)
# given a normal vector v of parameters at the boundary location x
# Return a callable function model_red(phi_red, x) where size(phi_red) = size(phi)-1
# but for the output y_red = model(phi_red, x):   (y_red-y)^2 ~ 0

function Reduce_model(model::Function, phi_boundary::AbstractArray, v_boundary::AbstractArray, data_dim::Tuple; model_id=0)
    # Create symbolic vectors for y and x
    string_as_symb_vec("phi", size(v_boundary)) # Dimension is number of parameters
    string_as_symb_vec("x", data_dim) # Dimension of input data dimension (for voltage and [Ca2+] is 2)
    
    # Get the symbolic model
    y_symb = model(phi, Ca=[x[1]], V=[x[2]], model_id=model_id) # Just do model(phi, x, model_id=model_id) in general
    
    # Given the normal at the boundary, substitute parameter combinations
    [y_comb, phi_comb] = phi_combine(y_symb, v)
    
    # Take the limits of the combined model
    y_lims = take_limits(y_comb, v)
    
    # Substitute back parameter combinations
    [y_red, phi_red, phi_red_mapping] = phi_reduce(y_lims, phi_comb)
    
    # Turn the symbolic model into a callable function
    model_red = create_reduced_model(y_red)
    
    # Compute the reduced parameter vector numerically
    phi_red_numeric = numeric_reduction(phi_boundary, phi_red_mapping)
    
    return y_red, phi_red_numeric 
    
end

Reduce_model (generic function with 1 method)

In [23]:
using NBInclude
nbinclude("BK_functions/BK_functions.ipynb")



LoadError: LoadError: UndefVarError: BK_simu not defined
while loading In[23], in expression starting on line 4

In [24]:
using JLD
@load "BK_results/geodesic_iter_320160705T12120800.jld"


16-element Array{Symbol,1}:
 symbol("#125#wsession")
 symbol("#135#wsession")
 symbol("#53#wsession") 
 symbol("#59#wsession") 
 :A                     
 :D                     
 :V                     
 :ans                   
 :cost                  
 :v                     
 :x                     
 :x_0                   
 :x_red1                
 :x_red2                
 :x_red3                
 :y                     

In [34]:
y_orig = Reduce_model(BK_simulator, x, v, size([0,-0.1]), model_id=12)

1x1 Array{Any,2}:
 (phi1*phi5*exp(38.1727608645031*phi2*x2) + phi1*phi4*phi5*phi6*x1*exp(38.1727608645031*phi2*x2)/phi3 + phi4*x1/phi3)^4/((phi1*exp(38.1727608645031*phi2*x2) + 1 + x1/phi3)^4 + (phi1*phi5*exp(38.1727608645031*phi2*x2) + phi1*phi4*phi5*phi6*x1*exp(38.1727608645031*phi2*x2)/phi3 + phi4*x1/phi3)^4)

In [35]:
y_orig[1]

                                                                              
                       /                                                      
                       |           38.1727608645031*phi2*x2   phi1*phi4*phi5*p
                       |phi1*phi5*e                         + ----------------
                       \                                                      
------------------------------------------------------------------------------
                                                                              
                                           4   /                              
/      38.1727608645031*phi2*x2        x1 \    |           38.1727608645031*ph
|phi1*e                         + 1 + ----|  + |phi1*phi5*e                   
\                                     phi3/    \                              

                                           4                        
        38.1727608645031*phi2*x2          \                  